# SparkSQL  

Comment construire un DataFrame à partir de sources diverses?  

How to create a DataFrame from different sources?


In [1]:
# "numpy" est une extension destinée à manipuler des matrices ou tableaux 
#    multidimensionnels ainsi que des fonctions mathématiques sur ces tableaux.
# "numpy" is the fundamental package for scientific computing with Python. It
#    contains a powerful N-dimensional array object, useful linear algebra.
import numpy as np

# "math" est une extension qui fournit des fonctions mathématiques définies par C standard.
# Package math provides access to the mathematical functions defined by the C standard.
import math

# On vérifie qu'il existe un Spark Context 
# We test if Spark Context existed
# from pyspark import SparkContext # Si il n'existe pas déjà / if it doesn't exist
#sc = SparkContext(appName="PythonSparkContext")
print(sc)

## A partir de fichiers JSON - From JSON files to DataFrame 

#### Créer SQLContext -  Build SQLContext

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#### Lire le fichier  - Read file

In [3]:
data = sqlContext.read.json("hdfs:///user/ecoles/test_Helene/file.json").cache()
# data = sqlContext.read.json("hdfs://ecoles.node1.pro.hupi.loc/user/ecoles/test_Helene/file.json").cache()

In [4]:
# "data" est un Dataframe
# "data" is a Dataframe
type(data)

pyspark.sql.dataframe.DataFrame

In [5]:
data.take(5)

[Row(CP=64600, age=25, nom=u'Smith', prenom=u'John'),
 Row(CP=59000, age=24, nom=u'Kelly', prenom=u'Stuart'),
 Row(CP=64000, age=20, nom=u'Kerry', prenom=u'John'),
 Row(CP=49000, age=28, nom=u'Smith', prenom=u'Anne'),
 Row(CP=64600, age=22, nom=u'Leclerc', prenom=u'Henry')]

In [6]:
# Selectionner la premiere ligne de data
# Select first row of data
data.first()

Row(CP=64600, age=25, nom=u'Smith', prenom=u'John')

In [7]:
# Selection des 3 premières lignes de data
# Select the 3 first rows of data
data.take(3)

[Row(CP=64600, age=25, nom=u'Smith', prenom=u'John'),
 Row(CP=59000, age=24, nom=u'Kelly', prenom=u'Stuart'),
 Row(CP=64000, age=20, nom=u'Kerry', prenom=u'John')]

#### Afficher le dataframe - Display the content of the DataFrame  

In [8]:
data.show()

+-----+---+-------+------+
|   CP|age|    nom|prenom|
+-----+---+-------+------+
|64600| 25|  Smith|  John|
|59000| 24|  Kelly|Stuart|
|64000| 20|  Kerry|  John|
|49000| 28|  Smith|  Anne|
|64600| 22|Leclerc| Henry|
|75013| 26| Robert| Alice|
|75004| 20| Dupont|Pierre|
+-----+---+-------+------+



#### Afficher les noms de colonnes - Print  the schema of column names

In [9]:
data.printSchema()

root
 |-- CP: long (nullable = true)
 |-- age: long (nullable = true)
 |-- nom: string (nullable = true)
 |-- prenom: string (nullable = true)



#### Faire des requetes sur un dataframe - Run queries on DataFrame

In [10]:
# Sélectionner la colonne "nom" 
# Select only the "nom" column
name = data.select("nom")

In [11]:
print("Le résultat est - The result is ")
name.show()

Le résultat est - The result is 
+-------+
|    nom|
+-------+
|  Smith|
|  Kelly|
|  Kerry|
|  Smith|
|Leclerc|
| Robert|
| Dupont|
+-------+



In [12]:
# Sélectionner deux colonnes "nom" et "prenom"
# Select two columns "nom" and "prenom"
fullname = data.select(data['nom'], data['prenom'])

In [13]:
print("Le résultat est - The result is ")
fullname.show()

Le résultat est - The result is 
+-------+------+
|    nom|prenom|
+-------+------+
|  Smith|  John|
|  Kelly|Stuart|
|  Kerry|  John|
|  Smith|  Anne|
|Leclerc| Henry|
| Robert| Alice|
| Dupont|Pierre|
+-------+------+



In [14]:
# Sélectionner les personnes ayant plus de 22 ans
# Select people who is older than 22 years old
val = data.filter(data['age'] > 22)

In [15]:
print("Le résultat est - The result is ")
val.show()

Le résultat est - The result is 
+-----+---+------+------+
|   CP|age|   nom|prenom|
+-----+---+------+------+
|64600| 25| Smith|  John|
|59000| 24| Kelly|Stuart|
|49000| 28| Smith|  Anne|
|75013| 26|Robert| Alice|
+-----+---+------+------+



In [16]:
# Pour faire des requètes SQL il faut passer par cette étape 
# To run queries SQL, we must follow this step
data.registerTempTable("data")

In [17]:
# Sélectionner la colonne "CP" 
# Select column "CP" 
cp = sqlContext.sql("select CP from data")

In [18]:
cp.collect()

[Row(CP=64600),
 Row(CP=59000),
 Row(CP=64000),
 Row(CP=49000),
 Row(CP=64600),
 Row(CP=75013),
 Row(CP=75004)]

## A partir de csv, txt - From csv, txt to DataFrame 

* La lecture d'un fichier csv ou txt est un peu différente que celle d'un JSON car il faut bien préciser le nom des colonnes du fichier texte.


* Read a text file is a little different from JSON file because we need to determine columns names in these file.

####  Importer SQLContext et types de data - Import SQLContext and data types

In [19]:
from pyspark.sql import SQLContext, Row

#### Create SQLContext - Build SQLContext 

In [20]:
#sqlContext = SQLContext(sc)

#### Lire le fichier et convertir chaque phrase en  tuple - Read text file and convert each line to a tuple

In [21]:
lines = sc.textFile("hdfs:///user/ecoles/test_Helene/file1.txt",use_unicode=False).cache()
parts = lines.map(lambda l: l.split(","))
base = parts.map(lambda p : Row(CP=p[0], ville=p[1]))

In [22]:
# "base" est un RDD ( PipelinesRDD est un RDD avec des étapes à effectuer (map) )
# "base" is a RDD ( Pipeline is a RDD with some jobs to do on it (map) )
type(base)

pyspark.rdd.PipelinedRDD

In [23]:
base.take(3)

[Row(CP='44000', ville=' Nantes'),
 Row(CP='49000', ville=' Angers'),
 Row(CP='50000', ville=' Baudre')]

* Il nous reste à convertir un RDD en DataFrame

* It remains to us to convert a RDD to a DataFrame

#### Créer un DataFrame - Create DataFrame

In [24]:
schemaData = sqlContext.createDataFrame(base)

In [25]:
# On a schemaData qui est un DataFrame
# We have schemaData as a DataFrame
type(schemaData)

pyspark.sql.dataframe.DataFrame

In [26]:
schemaData.printSchema()

root
 |-- CP: string (nullable = true)
 |-- ville: string (nullable = true)



In [27]:
# Pour faire des requetes SQL il faut passer à cette étape 
# To run queries SQL, we must follow this step
schemaData.registerTempTable("base")

* Maintenant, on pourra faire des requetes SQL sur "base"

* Now, we can execute SQL queries on "base"

#### Faire des requêtes sur un dataframe - Run queries on DataFrame

In [28]:
dat = sqlContext.sql("SELECT * FROM base")

In [29]:
# le resultat quand on applique requete SQl sur un DataFrame est aussi un DataFrame
# By running SQL queries on DataFrame, we will also have a DataFrame
type(dat)

pyspark.sql.dataframe.DataFrame

In [30]:
dat.show()

+-----+------------------+
|   CP|             ville|
+-----+------------------+
|44000|            Nantes|
|49000|            Angers|
|50000|            Baudre|
|51100|             Reims|
|59000|             Lille|
|64500| Saint-Jean de Luz|
|64600|            Anglet|
|75013|          Paris 13|
|75004|           Paris 4|
+-----+------------------+



In [31]:
data

DataFrame[CP: bigint, age: bigint, nom: string, prenom: string]

In [32]:
base

PythonRDD[47] at RDD at PythonRDD.scala:43

## Jointure entre un fichier JSON et un fichier txt - Join a JSON file and a txt file

* On peut faire la jointure entre le fichier "file.json" et le fichier "file1.txt" pour retrouver le nom de ville en convertissant les deux fichiers en DataFrame puis on fait la jointure.


* We can find city's names by converting two files in DataFrame, then do a join.


In [33]:
sqlContext.sql("select ville from base")

DataFrame[ville: string]

In [34]:
sqlContext.sql("select * from data")

DataFrame[CP: bigint, age: bigint, nom: string, prenom: string]

In [35]:
table = sqlContext.sql("SELECT data.nom, data.prenom, data.age, base.ville FROM data JOIN base ON data.CP = base.CP")

In [36]:
table.collect()

[Row(nom=u'Smith', prenom=u'Anne', age=28, ville=u' Angers'),
 Row(nom=u'Kelly', prenom=u'Stuart', age=24, ville=u' Lille'),
 Row(nom=u'Smith', prenom=u'John', age=25, ville=u' Anglet'),
 Row(nom=u'Leclerc', prenom=u'Henry', age=22, ville=u' Anglet'),
 Row(nom=u'Robert', prenom=u'Alice', age=26, ville=u' Paris 13'),
 Row(nom=u'Dupont', prenom=u'Pierre', age=20, ville=u' Paris 4')]

## DataFrame to RDD 

* Pour convertir un DataFrame en RDD, on fait un "map".


* To transform a DataFrame to a RDD, we must use function "map".

In [37]:
# Selectionner premiere colonne dans "data" et la convertir en RDD 
# Select first column in "data" and transform it to a RDD
rdd = dat.map(lambda x : (x[0], x[1]))

In [38]:
type(rdd)

pyspark.rdd.PipelinedRDD

In [39]:
rdd.collect()

[(u'44000', u' Nantes'),
 (u'49000', u' Angers'),
 (u'50000', u' Baudre'),
 (u'51100', u' Reims'),
 (u'59000', u' Lille'),
 (u'64500', u' Saint-Jean de Luz'),
 (u'64600', u' Anglet'),
 (u'75013', u' Paris 13'),
 (u'75004', u' Paris 4')]

In [40]:
type(rdd.collect())

list

In [41]:
rdd.map( lambda l : (l[0].encode("utf8") , l[1].encode("utf8")) ) \
   .collect()

[('44000', ' Nantes'),
 ('49000', ' Angers'),
 ('50000', ' Baudre'),
 ('51100', ' Reims'),
 ('59000', ' Lille'),
 ('64500', ' Saint-Jean de Luz'),
 ('64600', ' Anglet'),
 ('75013', ' Paris 13'),
 ('75004', ' Paris 4')]